# Overview  
### This notebook scrapes a list of cram school locations from the official website using web automation techniques.

In [39]:
#Playwrightを導入
import os
import random
import time
from playwright.async_api import async_playwright,expect

In [40]:
#ディレクトリを作成
os.makedirs('/Users/yuta-uebayashi/Desktop/Lede課題一覧/Lede個人プロジェクト/project3/cram_school', exist_ok=True)

In [41]:
async def open_browser(headless=False):
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless=headless)
    context = await browser.new_context()
    page = await browser.new_page()
    return browser, page

In [42]:
browser, page = await open_browser()

In [43]:
url = 'https://www.toshin.com/map/?top=1'
await page.goto(url)

TimeoutError: Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://www.toshin.com/map/?top=1", waiting until "load"


In [44]:
# 検索ボタンのXPATHを指定
xpath_search_botton = '//div[@class="clear-all"]/preceding-sibling::div[1]/input[@type="submit" and @value="検索"]'

In [45]:
search_button = page.locator(xpath_search_botton)

In [46]:
await search_button.click()

In [47]:
#セーブ
source = await page.content()
with open('/Users/yuta-uebayashi/Desktop/Lede課題一覧/Lede個人プロジェクト/project3/cram_school/cram_schools.html', 'w') as f:
    f.write(source)

In [48]:
# 「次ページ」のXPATHを指定
xpath_next = '//div[@id="result-next"]'

In [49]:
# 表のXPATHを指定
xpath_result = '//table'

In [50]:
next_button = page.locator(xpath_next)

In [51]:
await next_button.click()

In [52]:
# 最初のページを保存
i = 1
source = await page.content()
file_path = f'/Users/yuta-uebayashi/Desktop/Lede課題一覧/Lede個人プロジェクト/project3/cram_school/page_{i}.html'
with open(file_path, 'w') as f:
    f.write(source)

# 次のページ以降を繰り返し取得
collect = True

while collect:
    try:
        await expect(page.locator(xpath_next)).to_be_visible(timeout=1000)
        next_button = page.locator(xpath_next)
        await next_button.click()

        await page.locator(xpath_result).is_visible()

        i += 1  # ページ番号を更新（1ページ目は保存済み）

        source = await page.content()
        file_path = f'/Users/yuta-uebayashi/Desktop/Lede課題一覧/Lede個人プロジェクト/project3/cram_school/page_{i}.html'
        with open(file_path, 'w') as f:
            f.write(source)

    except Exception as e:
        print(f"停止しました: {e}")
        collect = False
        break

停止しました: Locator expected to be visible
Actual value: hidden 
Call log:
  - Expect "to_be_visible" with timeout 1000ms
  - waiting for locator("//div[@id=\"result-next\"]")
    5 × locator resolved to <div id="result-next" onclick="showNext( 15,972 );">次のページ</div>
      - unexpected value "hidden"



## Extract Data for All Organizations Using Pandas

In [62]:
import pandas as pd
import glob

# ファイルのパスをまとめて取得
files = sorted(glob.glob('/Users/yuta-uebayashi/Desktop/Lede課題一覧/Lede個人プロジェクト/project3/cram_school/page*.html'))

# データを格納するリスト
all_tables = []

for fn in files:
    tables = pd.read_html(fn)  # 各HTML内のすべてのテーブルを抽出
    if tables:  # 少なくとも1つあれば
        df = tables[0]  # 通常は1ページに1つだけ
        all_tables.append(df)

# すべてのテーブルを1つのDataFrameに結合
full_df = pd.concat(all_tables, ignore_index=True)

# 確認
print(full_df.shape)
display(full_df.head())

(972, 2)


,校舎名,住所
0,東進ハイスクール市ヶ谷校 (現役生対象),〒102-0076 東京都 千代田区 五番町5-1 JS市ヶ谷ビルB1F
1,東進ハイスクール人形町校 (現役生対象),〒103-0012 東京都 中央区 日本橋堀留町1-8-12
2,東進ハイスクール勝どき駅上校 (現役生対象),〒104-0054 東京都 中央区 勝どき1-7-1 勝どきサンスクェア3F
3,東進ハイスクール茗荷谷校 (現役生対象),〒112-0006 東京都 文京区 小日向4-6-12 茗荷谷駅MFビル（駅ビル）4F
4,東進ハイスクール本郷三丁目校 (現役生対象),〒113-0033 東京都 文京区 本郷3-34-3 本郷第一ビル6F


In [63]:
# 列名を変更
full_df.columns = ['Campus Name', 'Address']

In [64]:
full_df.head()

,Campus Name,Address
0,東進ハイスクール市ヶ谷校 (現役生対象),〒102-0076 東京都 千代田区 五番町5-1 JS市ヶ谷ビルB1F
1,東進ハイスクール人形町校 (現役生対象),〒103-0012 東京都 中央区 日本橋堀留町1-8-12
2,東進ハイスクール勝どき駅上校 (現役生対象),〒104-0054 東京都 中央区 勝どき1-7-1 勝どきサンスクェア3F
3,東進ハイスクール茗荷谷校 (現役生対象),〒112-0006 東京都 文京区 小日向4-6-12 茗荷谷駅MFビル（駅ビル）4F
4,東進ハイスクール本郷三丁目校 (現役生対象),〒113-0033 東京都 文京区 本郷3-34-3 本郷第一ビル6F


In [66]:
# CSVに保存
full_df.to_csv('/Users/yuta-uebayashi/Desktop/Lede課題一覧/Lede個人プロジェクト/project3/cram_school/all_cram_schools.csv', index=False)

## Extract prefecture and city names from addresses

In [ ]:
import pandas as pd
import re

# CSV読み込み（パスを変更してください）
df = pd.read_csv('/Users/yuta-uebayashi/Desktop/Lede課題一覧/Lede個人プロジェクト/project3/cram_school/all_cram_schools.csv')

# 郵便番号をスキップして、都道府県と市区町村だけ抽出（郡は削除、スペースなし）
def extract_region(address):
    # 郵便番号を削除
    address = re.sub(r'^〒?\d{3}-\d{4}\s*', '', address.strip())

    # 都道府県と市区町村の抽出
    match = re.match(r'(東京都|北海道|(?:京都|大阪)府|.{2,3}県)\s+(.+?[市区町村])', address)
    if match:
        pref = match.group(1)
        city = match.group(2)
        # 郡のみ除去（市・区・町・村は残す）
        city_clean = re.sub(r'^.+郡', '', city)
        return f"{pref}{city_clean}"
    else:
        return ""

# 適用して "Region" 列を作成
df["Region"] = df["Address"].apply(extract_region)

# 確認
print(df[["Address", "Region"]].head())

In [70]:
df.to_csv('/Users/yuta-uebayashi/Desktop/Lede課題一覧/Lede個人プロジェクト/project3/cram_school/cleaned_all_cram_schools.csv', index=False)